In [ ]:
import pandas as pd
import openai
import time
import io
import os 
from dotenv import load_dotenv
pd.set_option('display.max_colwidth', None)

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
client = openai.OpenAI(api_key=api_key)

# ABNAMRO csv processing 
remove credit card payments, pre-categorize income, savings, and investments

In [21]:
csv_path = "data/raw/bank_account/2024Oct_2025Mar_abnamro.csv"
batch_size = 20

df = pd.read_csv(csv_path, index_col=False)
df['description'] = df['description'].fillna("")
if 'category' not in df.columns:
    df['category'] = None

# get rid of credit card bills (this will be processed separately)
df = df[~df.description.str.contains('NL13ZZZ332005960000')]

# categorize income, savings, and investments 
def categorizeRow(row): 
    if row.amount > 2500: 
        return 'Income'
    elif 'direct sparen' in row.description.lower(): 
        return 'Savings'
    elif 'NL65ABNA0880878118' in row.description.upper(): 
        return 'Investments'
    elif 'yf wong' in row.description.lower(): 
        return 'Rent'
    elif row.amount > 0: 
        return 'Reimbursements' 
    else: 
        return row.category  # preserve existing
df['category'] = df.apply(lambda row: categorizeRow(row), axis=1)
df.rename(columns={'transactiondate': 'date'}, inplace=True)
df = df[['date', 'amount', 'description', 'category']]
df.to_csv('data/processed/bankStatements_2024Oct_2025Mar_abnamro.csv', index=False)
df.head()

,date,amount,description,category
0,20241001,-1067.70,SEPA Incasso algemeen doorlopend Incassant: NL19ZZZ271390540000 Naam: De Bruyn en Tak Makelaardi j B.V. Machtiging: 905-0000000042 Omschrijving: BETALINGSKENMERK: 3905000000430005 -CODE 000004-00 71-00000043 IBAN: NL13INGB0105652768,None
1,20241002,-20.00,SEPA Incasso algemeen doorlopend Incassant: NL03ZZZ301771260000 Naam: TLS BV inz. OV-Chipkaart Machtiging: NL03ZZZ301771260000ARAAU82000944343 Omschrijving: Automatisch opladen OV-chipkaart Kaartnummer:35280 53445814037 Oplaaddatum/tijd:27-09-24 16:07,None
2,20241002,-20.00,SEPA Incasso algemeen doorlopend Incassant: NL03ZZZ301771260000 Naam: TLS BV inz. OV-Chipkaart Machtiging: NL03ZZZ301771260000ARAAU82000944343 Omschrijving: Automatisch opladen OV-chipkaart Kaartnummer:35280 53445814037 Oplaaddatum/tijd:28-09-24 16:12,None
3,20241002,3016.96,SEPA Overboeking IBAN: BE48967056780227 BIC: TRWIBEB1XXX Naam: TransferWise Omschrijving: TANYA PUI YUNG TSU I Kenmerk: 1064438267 ID debiteur: P9209534,Income
4,20241002,-9.50,"BEA, Betaalpas CCV*PSU BV,PAS033 NR:CT664993, 02.10.24/12:41 AMSTERDAM",None


# Extracting data from credit card pdf statements

In [ ]:
class PdfStatementExtractor: 
    
    def __init__(self, folder_path, csv_path): 
        self.folder_path = folder_path 
        self.csv_path = csv_path

    def run(self): 
        self.deleteOldCsv() 
        self.createAssistant()
        self.getFileNames() 
        for file_name in self.file_names: 
            self.messageAssistant(file_name)
            self.runAssistant()
            self.recordOutput() 

    def deleteOldCsv(self): 
        file_path = self.csv_path 
        if os.path.exists(file_path):
            os.remove(file_path)

    def createAssistant(self): 
        self.assistant = client.beta.assistants.create(
            name="PDF Extractor",
            instructions=(
                "You are a helpful assistant that extracts tables from credit card statement PDFs "
                "and returns them in CSV format."
            ),
            model="gpt-4o", 
            tools=[{"type": "file_search"}],
        )

    def getFileNames(self): 
        folder_path = self.folder_path
        self.file_names = [os.path.join(folder_path, f) for f in os.listdir(folder_path)
                           if os.path.isfile(os.path.join(folder_path, f))]

    def messageAssistant(self, file_name): 
        print(f'processing {file_name}...')
        message_file = client.files.create(
        file=open(file_name, "rb"), purpose="assistants"
        )

        self.thread = client.beta.threads.create(
        messages=[
            {
            "role": "user",
            "content": (
                            "This is a credit card statement. Extract all transactions as a table with columns: "
                            "['date', 'description', 'amount'].\n\n"
                            "change the transactiondate values to DD/MM/YY format. "
                            "Change the amount values into numbers (floats) - with 'bij' representing a positive number, and 'af' a negative number"
                            "Return the transactions in plain CSV format, including the header row. Enclose all fields in double quotes."
                            "Do not add explanations or formatting (such as ``````), just output the CSV text."
                        ),
            # Attach the new file to the message.
            "attachments": [
                { "file_id": message_file.id, "tools": [{"type": "file_search"}] }
            ],
            }
        ]
        )

    def runAssistant(self): 
        run = client.beta.threads.runs.create(
            thread_id=self.thread.id,
            assistant_id=self.assistant.id
        )

        while True:
            run_status = client.beta.threads.runs.retrieve(thread_id=self.thread.id, run_id=run.id)
            if run_status.status == "completed":
                break
            elif run_status.status in ["failed", "cancelled", "expired"]:
                raise Exception(f"Run failed with status: {run_status.status}")
            time.sleep(2)

        messages = client.beta.threads.messages.list(thread_id=self.thread.id)
        self.output = messages.data[0].content[0].text.value
        # print("GPT output:\n", output)  # Optional: see what GPT returned

    def recordOutput(self): 
        csv_path = self.csv_path
        df = pd.read_csv(io.StringIO(self.output))
        if os.path.exists(csv_path): 
            df.to_csv(csv_path, mode='a', index=False, header=False)
        else: 
            df.to_csv(csv_path, mode='w', index=False, header=True)
        print(f'results saved to {csv_path}')


In [13]:
folder_path = 'data/raw/credit_card'
csv_path = 'data/processed/creditCardStatements.csv'
PdfStatementExtractor(folder_path, csv_path).run()

processing data/raw/credit_card/Statement-71583590014-2025-01.pdf...
results saved to data/processed/creditCardStatements.csv
processing data/raw/credit_card/Statement-71583590014-2025-03.pdf...
results saved to data/processed/creditCardStatements.csv
processing data/raw/credit_card/Statement-71583590014-2025-02.pdf...
results saved to data/processed/creditCardStatements.csv
processing data/raw/credit_card/Statement-71583590014-2024-10.pdf...
results saved to data/processed/creditCardStatements.csv
processing data/raw/credit_card/Statement-71583590014-2024-11.pdf...
results saved to data/processed/creditCardStatements.csv
processing data/raw/credit_card/Statement-71583590014-2024-12.pdf...
results saved to data/processed/creditCardStatements.csv


# Combining credit card and bank statements

In [26]:
# clean credit card statements 
df = pd.read_csv('data/processed/creditCardStatements.csv')
df = df[df.amount < 0]
df['category'] = None
df['date'] = pd.to_datetime(df.date, format='%d/%m/%y')
df = df[['date', 'description', 'amount', 'category']]
credit_card = df.copy()

# bank statements 
df = pd.read_csv('data/processed/bankStatements_2024Oct_2025Mar_abnamro.csv')
df['date'] = pd.to_datetime(df.date, format='%Y%m%d')
df = df[['date', 'description', 'amount', 'category']]
bank = df.copy() 

combined = pd.concat([credit_card, bank])
combined.to_csv('data/processed/statements_combined.csv', index=False)

# Categorizing rows with chatGPT 

In [ ]:
import openai
import pandas as pd
import json
import time
import os
from openai import OpenAI
from dotenv import load_dotenv

In [ ]:
class BankStatementCategoriser: 

    def __init__(self, bank_statement_path, results_path, assistant_instructions):
        load_dotenv()
        api_key = os.getenv("OPENAI_API_KEY")
        self.client = OpenAI(api_key=api_key)
        self.gpt_model = 'gpt-4o'
        self.assistant_instructions = assistant_instructions 

        self.bank_statement_path = bank_statement_path
        self.results_path = results_path 
        
        self.df = pd.read_csv(self.bank_statement_path) 
        self.batch_size = 20 

    def run(self): 
        self.delete_old_csv()
        self.create_assistant()
        for i in range(0, len(self.df), self.batch_size):
            self.get_descriptions(i)
            self.process_categorised_descriptions(i)

    def delete_old_csv(self): 
        file_path = self.results_path 
        if os.path.exists(file_path):
            os.remove(file_path)

    def create_assistant(self): 
        self.assistant = self.client.beta.assistants.create(
            name="Budget Categorisation Assistant",
            instructions=open(self.assistant_instructions).read(),
            model=self.gpt_model
        )

    def get_descriptions(self, i): 
        batch = self.df.iloc[i:i+self.batch_size]
        if batch['category'].notna().all():
            return # skip further processing 
        print(f"Processing rows {i}–{i + self.batch_size - 1} of {len(self.df)}")
        self.descriptions = batch['description'].tolist()

    def process_categorised_descriptions(self, i): 
        df = self.df 
        descriptions = self.descriptions

        try:
            categories = self.batch_categorize(descriptions)
            if len(categories) != len(descriptions):
                print("❌ Mismatch in returned categories, skipping this batch.")
                print(f'n descriptions: {len(descriptions)} - {descriptions}')
                print(f'n categories: {len(categories)} - {categories}')
                return

            df.loc[i:i+self.batch_size-1, 'category'] = categories
            df.to_csv(self.results_path, index=False)
            print(f"✅ Saved rows {i}–{i + self.batch_size - 1}")

        except Exception as e:
            print(f"❌ Error in batch {i}–{i + self.batch_size - 1}: {e}")
            return

    def batch_categorize(self, descriptions):
        # Number the descriptions to guide GPT
        numbered_descriptions = "\n".join([f"{i+1}. {desc}" for i, desc in enumerate(descriptions)])

        self.thread = self.client.beta.threads.create()
        self.client.beta.threads.messages.create(
            thread_id=self.thread.id,
            role="user",
            content=f'Descriptions: \n{numbered_descriptions}'
        )
        run = self.client.beta.threads.runs.create(
            thread_id=self.thread.id,
            assistant_id=self.assistant.id
        )
        while True:
            run_status = self.client.beta.threads.runs.retrieve(
                thread_id=self.thread.id,
                run_id=run.id
            )
            if run_status.status == "completed":
                break
            elif run_status.status in ["failed", "cancelled", "expired"]:
                raise Exception(f"Run failed with status: {run_status.status}")
            time.sleep(1)  # avoid hammering the API

        messages = self.client.beta.threads.messages.list(thread_id=self.thread.id)
        content = messages.data[0].content[0].text.value

        # Parse lines manually
        lines = content.splitlines()
        categories = []

        for line in lines:
            if "." in line:
                try:
                    _, category = line.split(".", 1)
                    categories.append(category.strip())
                except ValueError:
                    continue

        if len(categories) != 20:
            print(f"❌ GPT returned {len(categories)} categories instead of 20.")

        return categories

In [79]:
bank_statement_path = 'data/processed/statements_combined.csv'
results_path = 'results/statements_categorised_241001_to_250331.csv'
assistant_instructions = 'data/processed/assistant_instructions.txt'
BankStatementCategoriser(bank_statement_path, results_path, assistant_instructions).run()

Processing rows 0–19 of 448
✅ Saved rows 0–19
Processing rows 20–39 of 448
✅ Saved rows 20–39
Processing rows 40–59 of 448
✅ Saved rows 40–59
Processing rows 60–79 of 448
✅ Saved rows 60–79
Processing rows 80–99 of 448
✅ Saved rows 80–99
Processing rows 100–119 of 448
✅ Saved rows 100–119
Processing rows 120–139 of 448
✅ Saved rows 120–139
Processing rows 140–159 of 448
✅ Saved rows 140–159
Processing rows 160–179 of 448
✅ Saved rows 160–179
Processing rows 180–199 of 448
✅ Saved rows 180–199
Processing rows 200–219 of 448
✅ Saved rows 200–219
Processing rows 220–239 of 448
✅ Saved rows 220–239
Processing rows 240–259 of 448
✅ Saved rows 240–259
Processing rows 260–279 of 448
✅ Saved rows 260–279
Processing rows 280–299 of 448
✅ Saved rows 280–299
Processing rows 300–319 of 448
✅ Saved rows 300–319
Processing rows 320–339 of 448
✅ Saved rows 320–339
Processing rows 340–359 of 448
✅ Saved rows 340–359
Processing rows 360–379 of 448
✅ Saved rows 360–379
Processing rows 380–399 of 448
✅ S

# Analyze expenses

In [4]:
import pandas as pd
import plotly.express as px

In [5]:
df = pd.read_csv('results/statements_categorised_241001_to_250331.csv')

# print n uncategorised rows 
n_na_rows = len(df[df.category.isna()])
perc_na = round(n_na_rows / len(df) * 100) 
print(f'{perc_na}% of rows were not categorised')

# adjust df for visualisation 
def categorizeRow(row): 
    if row.amount > 2500: 
        return 'Income'
    elif 'direct sparen' in row.description.lower(): 
        return 'Savings'
    elif 'NL65ABNA0880878118' in row.description.upper(): 
        return 'Investments'
    elif 'yf wong' in row.description.lower(): 
        return 'Rent'
    elif 'belastingdienst' in row.description.lower(): 
        return 'Taxes'
    elif 'NUS ECOLL2.0 6565168238 SGP' in row.description.upper(): 
        return 'to remove' # conference cost for Singapore, was reimbursed 
    elif row.amount > 0: 
        return 'Reimbursements' 
    else: 
        return row.category  # preserve existing
df['category'] = df.apply(lambda row: categorizeRow(row), axis=1)
category_totals = df.groupby('category', as_index=False)['amount'].sum()
category_totals = category_totals[category_totals.amount < 0]
category_totals = category_totals[category_totals.category != 'to remove']
category_totals = category_totals[~category_totals.category.isin(['Investments', 'Savings'])]
category_totals['amount'] = category_totals['amount'].abs()

# adjust taxes - they are paid annually, so should be split accordingly 
n_months = 6 # 1-Oct-2024 till 31-Mar-2025
def taxes(row): 
    if row.category == 'Taxes': 
        row.amount = row.amount * n_months / 12 # reduce tax amount proportionally to n months for study period 
        return row 
    else: 
        return row 
category_totals = category_totals.apply(lambda row: taxes(row), axis=1)

# calculate average monthly spending 
n_months = 6 # 1-Oct-2024 till 31-Mar-2025
total_spending = category_totals.amount.sum() 
av_spending_per_month = round(total_spending/n_months)
print(f'total spending: {total_spending}. Average spending over {n_months} months: {av_spending_per_month}')

# visualise
category_totals = category_totals.sort_values('amount')
category_totals['amount'] = category_totals.amount / n_months
fig = px.bar(
    category_totals,
    x='amount',
    y='category',
    orientation='h',
    title=f'av spending: {av_spending_per_month} per month',
    width=700
)

fig.show()

0% of rows were not categorised
total spending: 17994.779999999995. Average spending over 6 months: 2999


In [132]:
temp = df[(df.category == 'Shopping')]
# temp.apply(lambda row: print(f'{row.amount}\n{row.description}\n'), axis=1)
temp.description.unique()

array(['AMAZONRETAIL*AC3LV7BA5 WWW.AMAZON.NL LUX',
       'AMZN MKTP NL*J85GE8265 AMAZON.NL LUX',
       'AMAZONRETAIL*2L62S08L5 WWW.AMAZON.NL LUX',
       'AMZN MKTP NL*RI8AE3IQ4 LUXEMBOURG LUX',
       'AMZN MKTP NL*RZ5LV9TY4 AMAZON.NL LUX',
       'AMAZONRETAIL*LG5703U45 WWW.AMAZON.NL LUX',
       'AMAZONRETAIL*TK8HE1F24 WWW.AMAZON.NL LUX',
       'AMZN MKTP NL*RT1NI4TU4 AMAZON.NL LUX',
       'AMZN MKTP NL*RT6AZ6TY4 AMAZON.NL LUX',
       'AMZN MKTP NL*R800R9YK4 AMAZON.NL LUX',
       'AMAZONRETAIL*R01324A04 WWW.AMAZON.NL LUX',
       'AMZN MKTP NL*TO68O2254 AMAZON.NL LUX',
       'AMZN MKTP NL*TR2V54EP4 AMAZON.NL LUX',
       'AMZN MKTP CA*ZR5EA9VB1 AMAZON.CA USA',
       'AMZN MKTP CA*Z17YE3SA0 AMAZON.CA USA',
       'AMZN MKTP CA*Z96M74Y22 AMAZON.CA USA',
       'BEA, Betaalpas                   CCV*DE DIAMANTEN RING,PAS033    NR:CT471206, 06.10.24/10:05      DELFT',
       'BEA, Apple Pay                   CCV*HOME MIXTURE,PAS033         NR:CT469261, 07.10.24/17:02      DELFT',

In [102]:
temp = df[
    (~df.category.isin(['Travel', 'Rent', 'Savings', 'Insurance', 'Investments', 'Taxes', 'Utilities'])) & 
    (df.amount < -100)
    ]
def print_results(row): 
    print(f'{row.category}, {row.amount}')
    print(f'{row.description}\n')
temp.apply(lambda row: print_results(row), axis=1)

Books & Education, -1112.49
NUS ECOLL2.0 6565168238 SGP

Other, -209.0
TRUSTEDHOUSESITTERS.CO ENGLAND GBR

Subscriptions, -160.8
ROAMRESEARCH.COM OAKLAND USA

Subscriptions, -116.83
MURAL SOUTH SAN FRA USA

Shopping, -150.0
SEPA iDEAL                       IBAN: NL04ADYB2017400157        BIC: ADYBNL2A                    Naam: HOKA                      Omschrijving: B6MRVVKS4QJCRTF32N 1Y8 7180187928548837 Deckers    Kenmerk: 30-10-2024 20:11 718018 7928548837

Transport, -558.75
SEPA Incasso algemeen doorlopend Incassant: NL03ZZZ301243580000  Naam: NS GROEP IZ NS REIZIGERS   Machtiging: 200000623751        Omschrijving: Factuur: 451000720 339                             IBAN: NL14ABNA0576850675         Kenmerk: 240036857432           Voor: TPY TSUI

Restaurants & Cafés, -156.93
/TRTP/iDEAL/IBAN/NL31ABNA0494688556/BIC/ABNANL2A/NAME/. mai via Takeaway.com/REMI/W3T3Y8 7030376987309751 bestelling via Thuisbezorgd.nl O mai/EREF/15-01-2025 17:31 7030376987309751

Health & Hygiene, -142.5
/TRT

18     None
23     None
68     None
70     None
161    None
207    None
248    None
354    None
dtype: object